In [15]:
import os
import dask.dataframe as dd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from lightgbm import LGBMRegressor, LGBMClassifier
import optuna
import pyarrow.parquet as pq
import pandas as pd
import datetime
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error

print('Ok - import')

Ok - import


In [2]:
# ---------------- Przygotowanie danych ----------------
competition_path = "M:/PycharmProjects/ds-zt-LGBMRegressor-main/kaggle-dtset"  # Ścieżka do danych konkursowych
batch_size = 1000000  # Rozmiar batcha treningowych
percent_of_dataset = 0.075  # Ile procent dataset używamy - 7,5%

# Ścieżki do danych
train_path = os.path.join(competition_path, "train.parquet")
test_path = os.path.join(competition_path, "test.parquet")
features = [f'feature_{i:02}' for i in range(79)]
target_train = 'responder_6'  # Target do przewidzenia
weights = 'weight'

# Wczytanie danych przy użyciu Dask
ddf_train = dd.read_parquet(train_path, columns=features + [target_train, weights])
data_train = ddf_train.sample(frac=percent_of_dataset, random_state=42).compute()
data_train = data_train.dropna(subset=features + [target_train, weights])  # Usunięcie braków

X_train = data_train[features]
y_train = data_train[target_train]
sample_weights_train = data_train[weights]

print(f"Próbka danych: {data_train.shape}")


Próbka danych: (2652773, 81)


In [ ]:
# ---------------- Optymalizacja z Optuna ----------------
def objective(trial):
    # Przestrzeń hiperparametrów
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),           # Liczba drzew
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1), # Współczynnik uczenia
        'num_leaves': trial.suggest_int('num_leaves', 31, 100),                # Liczba liści
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.8, 1.0), # Frakcja cech
        'subsample': trial.suggest_uniform('subsample', 0.8, 1.0),             # Frakcja próbek
        'max_depth': trial.suggest_int('max_depth', -1, 15),                   # Maksymalna głębokość
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 50),   # Minimalna liczba próbek w liściu
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 10.0),        # Regularyzacja L1
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-3, 10.0),        # Regularyzacja L2
        'max_bin': trial.suggest_int('max_bin', 127, 255),                     # Maksymalna liczba binów
        'random_state': 42
    }

    # Model LightGBM z nowym celem
    model = LGBMRegressor(objective='regression_l1', metric='mae', **params)

    # Cross-validation z podziałem czasowym
    tscv = TimeSeriesSplit(n_splits=5)
    scores = []

    for train_idx, valid_idx in tscv.split(X_train):
        X_fold_train, X_fold_valid = X_train.iloc[train_idx], X_train.iloc[valid_idx]
        y_fold_train, y_fold_valid = y_train.iloc[train_idx], y_train.iloc[valid_idx]
        weights_fold_train = sample_weights_train.iloc[train_idx]

        # Dopasowanie modelu (bez verbose)
        model.fit(
            X_fold_train, y_fold_train, 
            sample_weight=weights_fold_train,
            eval_set=[(X_fold_valid, y_fold_valid)],
            eval_metric='mae',
        )

        # Predykcja i obliczenie MAE
        preds = model.predict(X_fold_valid)
        mae = mean_absolute_error(y_fold_valid, preds)
        scores.append(mae)

    # Zwracanie średniej wartości MAE
    return np.mean(scores)

# Optymalizacja z Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Wyświetlenie najlepszego zestawu parametrów i wyniku
print("\nNajlepsze parametry:")
print(study.best_params)

print("\nNajlepszy wynik (MAE):")
print(study.best_value)


[I 2025-01-28 00:02:07,588] A new study created in memory with name: no-name-d6594396-82ab-43a8-866b-26743ad95393


[LightGBM] [Warning] feature_fraction is set=0.8085912586389292, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8085912586389292
[LightGBM] [Warning] lambda_l1 is set=0.29203742098069024, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.29203742098069024
[LightGBM] [Warning] lambda_l2 is set=0.2549254896291711, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.2549254896291711
[LightGBM] [Warning] feature_fraction is set=0.8085912586389292, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8085912586389292
[LightGBM] [Warning] lambda_l1 is set=0.29203742098069024, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.29203742098069024
[LightGBM] [Warning] lambda_l2 is set=0.2549254896291711, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.2549254896291711
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.122630 seconds.
You can set `force_col_wise=true` to remove the ov

[I 2025-01-28 00:21:36,600] Trial 0 finished with value: 0.5132235036253691 and parameters: {'n_estimators': 397, 'learning_rate': 0.03523966114274594, 'num_leaves': 40, 'feature_fraction': 0.8085912586389292, 'subsample': 0.9173389905893674, 'max_depth': 10, 'min_child_samples': 23, 'lambda_l1': 0.29203742098069024, 'lambda_l2': 0.2549254896291711, 'max_bin': 199}. Best is trial 0 with value: 0.5132235036253691.


[LightGBM] [Warning] feature_fraction is set=0.915387021443812, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.915387021443812
[LightGBM] [Warning] lambda_l1 is set=0.002310417747884027, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.002310417747884027
[LightGBM] [Warning] lambda_l2 is set=2.6545280720671145, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.6545280720671145
[LightGBM] [Warning] feature_fraction is set=0.915387021443812, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.915387021443812
[LightGBM] [Warning] lambda_l1 is set=0.002310417747884027, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.002310417747884027
[LightGBM] [Warning] lambda_l2 is set=2.6545280720671145, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.6545280720671145
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.109895 seconds.
You can set `force_col_wise=true` to remove the ov

[I 2025-01-28 00:49:09,341] Trial 1 finished with value: 0.513252491408559 and parameters: {'n_estimators': 490, 'learning_rate': 0.05379221124871343, 'num_leaves': 31, 'feature_fraction': 0.915387021443812, 'subsample': 0.9456808272659598, 'max_depth': 11, 'min_child_samples': 47, 'lambda_l1': 0.002310417747884027, 'lambda_l2': 2.6545280720671145, 'max_bin': 198}. Best is trial 0 with value: 0.5132235036253691.


...

[LightGBM] [Warning] feature_fraction is set=0.8941591746345781, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8941591746345781
[LightGBM] [Warning] lambda_l1 is set=2.974012813507761, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.974012813507761
[LightGBM] [Warning] lambda_l2 is set=0.005752207426891498, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.005752207426891498
[LightGBM] [Warning] feature_fraction is set=0.8941591746345781, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8941591746345781
[LightGBM] [Warning] lambda_l1 is set=2.974012813507761, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.974012813507761
[LightGBM] [Warning] lambda_l2 is set=0.005752207426891498, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.005752207426891498
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.111180 seconds.
You can set `force_col_wise=true` to remove the ov

[I 2025-01-28 06:41:09,452] Trial 28 finished with value: 0.5132570642959968 and parameters: {'n_estimators': 270, 'learning_rate': 0.027433970044784536, 'num_leaves': 43, 'feature_fraction': 0.8941591746345781, 'subsample': 0.9539531494275383, 'max_depth': 6, 'min_child_samples': 43, 'lambda_l1': 2.974012813507761, 'lambda_l2': 0.005752207426891498, 'max_bin': 158}. Best is trial 2 with value: 0.5130985813282369.


[LightGBM] [Warning] feature_fraction is set=0.8480801238749307, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8480801238749307
[LightGBM] [Warning] lambda_l1 is set=1.3317372017993974, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.3317372017993974
[LightGBM] [Warning] lambda_l2 is set=0.23834633993416152, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.23834633993416152
[LightGBM] [Warning] feature_fraction is set=0.8480801238749307, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8480801238749307
[LightGBM] [Warning] lambda_l1 is set=1.3317372017993974, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.3317372017993974
[LightGBM] [Warning] lambda_l2 is set=0.23834633993416152, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.23834633993416152
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.098846 seconds.
You can set `force_col_wise=true` to remove the ov

[I 2025-01-28 06:55:43,145] Trial 29 finished with value: 0.513176149786513 and parameters: {'n_estimators': 318, 'learning_rate': 0.036412944214061875, 'num_leaves': 38, 'feature_fraction': 0.8480801238749307, 'subsample': 0.9102594569526127, 'max_depth': 9, 'min_child_samples': 24, 'lambda_l1': 1.3317372017993974, 'lambda_l2': 0.23834633993416152, 'max_bin': 193}. Best is trial 2 with value: 0.5130985813282369.


[LightGBM] [Warning] feature_fraction is set=0.9649638571010073, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9649638571010073
[LightGBM] [Warning] lambda_l1 is set=9.927540874147926, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.927540874147926
[LightGBM] [Warning] lambda_l2 is set=0.05676574808635271, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.05676574808635271
[LightGBM] [Warning] feature_fraction is set=0.9649638571010073, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9649638571010073
[LightGBM] [Warning] lambda_l1 is set=9.927540874147926, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.927540874147926
[LightGBM] [Warning] lambda_l2 is set=0.05676574808635271, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.05676574808635271
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.098945 seconds.
You can set `force_col_wise=true` to remove the overhe

In [4]:
# ---------------- Wyniki Optuna ----------------
print("\nNajlepsze parametry:")
print(study.best_params)

print("\nNajlepszy wynik (MSE):")
print(study.best_value)

# Zapis wyników do pliku
now = datetime.datetime.now()
date_str = now.strftime("%Y-%m-%d-%H-%M-%S")
file_name = f"optuna_results_{date_str}.txt"

with open(file_name, 'w') as f:
    f.write(f"Najlepsze parametry: {study.best_params}\n")
    f.write(f"Najlepszy wynik (MSE): {study.best_value:.4f}\n")

print(f"\nWyniki zapisano do pliku: {file_name}")


Najlepsze parametry:
{'n_estimators': 459, 'learning_rate': 0.033441061327066174, 'num_leaves': 85, 'feature_fraction': 0.8086691830052026, 'subsample': 0.9862178362318438, 'max_depth': 9, 'min_child_samples': 25, 'lambda_l1': 0.0049638961104124055, 'lambda_l2': 0.006126150398752788, 'max_bin': 254}

Najlepszy wynik (MSE):
0.6385294016276032

Wyniki zapisano do pliku: optuna_results_2025-01-27-16-10-22.txt


In [ ]:
from sklearn.metrics import f1_score
